In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.spatial import Voronoi, voronoi_plot_2d, KDTree
from scipy.ndimage import map_coordinates

from skimage.io import imread, imsave
from skimage import feature, filters, measure
from skimage.color import rgb2gray
from skimage.filters.rank import gradient
from skimage.transform import rescale, rotate
from matplotlib import pyplot as plt
from matplotlib.pyplot import Axes

In [ ]:
#image = imread("./data/input/42501f8a-5015-41e2-a87b-2980df8237a4.webp")/255.0
image = imread("./data/input/a4ca9877-c77b-4a8e-9d7a-aa621a7736fe.webp")/255.0
# image = imread("./data/input/carpet.webp")
# image = imread("./data/input/abstract-pattern-in-dark-pastel-tones-purple-background-and-green-blue-and-pink-spirals-vector.jpg")
#image = imread("./data/input/Cge6pcFRQLmteLQyGiIznQ.webp")
#image = rotate(image,90, resize=True)
plt.imshow(image)

In [ ]:
# image_width, image_height, _ = image.shape
NUMBER_OF_POINTS = 8000

In [ ]:
def draw_simulated_points(ax: Axes, heightmap: np.ndarray, points_xy: np.ndarray, show_voronoi=False):
    ax.imshow(heightmap)
    ax.scatter(x=points_xy[:, 1], y=points_xy[:, 0], s=0.4, c="red")
    if show_voronoi:
        voronoi_plot_2d(
            Voronoi(points_xy[:,::-1]),
            ax=ax,
            show_vertices=False,
            show_points=False,
            line_colors='red',
            line_alpha=0.5,
        )
    ax.set_xlim((0, heightmap.shape[1]))  # width
    ax.set_ylim((0, heightmap.shape[0]))  # height
    ax.axis("off")
    ax.invert_yaxis()
    return ax

In [ ]:
def adjust_points_via_forcefield(points, force_field_x, force_field_y, iterations=30, force_multiplier=50, repulsion_multiplier=10):
    """
    Adjusts points based on a simulated force field derived from an image.

    Parameters:
    - points: numpy.ndarray, the original points as an Nx2 array.
    - image: numpy.ndarray, the image used to compute the force field.
    - direction: str, 'up' to move points up the force field, 'down' to move them down.
    - iterations: int, number of iterations to apply the force field adjustments.
    - force_multiplier: float, multiplier for the force field effect.
    - repulsion_multiplier: float, multiplier for the repulsion effect.

    Returns:
    - numpy.ndarray, the adjusted points.
    """
    
    
    # Initialize points
    voronoi_point_xy = points.copy()

    for iteration in range(iterations):
        force_x = map_coordinates(force_field_x, voronoi_point_xy.transpose())
        force_y = map_coordinates(force_field_y, voronoi_point_xy.transpose())
        
        
        # Compute pairwise differences and repulsion forces as before
        diff = voronoi_point_xy[:, None, :] - voronoi_point_xy[None, :, :]
        dist_squared = np.sum(diff ** 2, axis=2)
        epsilon = 1e-5
        repulsion_strength = 1 / (dist_squared + epsilon)
        repulsion_strength[range(len(points)), range(len(points))] = 0
        repulsion_vectors = np.sum(diff * repulsion_strength[:, :, None] / np.sqrt(dist_squared + epsilon)[:, :, None], axis=1)
        
        # Apply the forces
        voronoi_point_xy += np.stack([force_x, force_y], axis=-1) * force_multiplier + repulsion_vectors * repulsion_multiplier
    
    return voronoi_point_xy

In [ ]:
voronoi_point_xy_original = np.random.rand(NUMBER_OF_POINTS, 2) * np.array(image.shape[:2])
voronoi_point_xy_light_seekers, voronoi_point_xy_dark_seekers = np.array_split(voronoi_point_xy_original,2)

# Ensure the input image is grayscale for the force field computation
simulated_heightmap = filters.difference_of_gaussians(rgb2gray(image), low_sigma=8)
# plt.imshow(simulated_heightmap)

# Compute the gradient (force field) of the height map
fx = filters.sobel_h(simulated_heightmap)
fy = filters.sobel_v(simulated_heightmap)
#force_field = np.stack([fx, fy], axis=-1)

voronoi_point_xy = np.concatenate([
    adjust_points_via_forcefield(voronoi_point_xy_light_seekers,iterations=30, force_field_x=fx, force_field_y=fy, force_multiplier= 80, repulsion_multiplier=20),
    adjust_points_via_forcefield(voronoi_point_xy_dark_seekers ,iterations=30, force_field_x=fx, force_field_y=fy, force_multiplier=-80, repulsion_multiplier=20),
])
#voronoi_point_xy = voronoi_point_xy_original

tree = KDTree(voronoi_point_xy)

x, y = np.meshgrid(range(image.shape[1]), range(image.shape[0]))
yx_indexes_flat = np.stack([y.flatten(), x.flatten()],axis=-1)

_, group_index = tree.query(yx_indexes_flat)

unique_groups, unique_groups_indexes = np.unique(group_index, return_inverse=True)

total_pixel_colors = np.zeros((*unique_groups.shape,3))
np.add.at(
    total_pixel_colors,
    unique_groups_indexes,
    image.reshape(-1,3)
)
total_pixel_counts = np.zeros(unique_groups.shape)
np.add.at(
    total_pixel_counts,
    unique_groups_indexes,
    1,
)

average_color = total_pixel_colors / total_pixel_counts.reshape(-1,1)
new_pixel_values = average_color[unique_groups_indexes].reshape(image.shape)

In [ ]:
fig, axs = plt.subplots(1,3)
draw_simulated_points(axs[0], image, voronoi_point_xy_original)
draw_simulated_points(axs[1], simulated_heightmap, voronoi_point_xy, show_voronoi=True)
axs[2].imshow(new_pixel_values)
axs[2].axis("off")
axs[2].set_xlim([0, new_pixel_values.shape[1]])
axs[2].set_ylim([0, new_pixel_values.shape[0]])
axs[2].invert_yaxis()
fig.set_size_inches(20,20)
plt.tight_layout()

In [ ]:
fig, axs = plt.subplots(1,2)
draw_simulated_points(axs[0], image, voronoi_point_xy_original)
#draw_simulated_points(axs[1], simulated_heightmap, voronoi_point_xy, show_voronoi=True)
axs[1].imshow(new_pixel_values)
axs[1].axis("off")
axs[1].set_xlim([0, new_pixel_values.shape[1]])
axs[1].set_ylim([0, new_pixel_values.shape[0]])
axs[1].invert_yaxis()
fig.set_size_inches(10,20)
plt.tight_layout()

In [ ]:

UPSCALE_FACTOR = 4  # Specify your desired upscale factor here

# Generate grid for upscaled image
x_upscaled, y_upscaled = np.meshgrid(range(image.shape[1]*UPSCALE_FACTOR), range(image.shape[0]*UPSCALE_FACTOR))
yx_indexes_flat_upscaled = np.stack([y_upscaled.flatten(), x_upscaled.flatten()], axis=-1) 

_, upscaled_group_index = tree.query(yx_indexes_flat_upscaled / UPSCALE_FACTOR)

upscaled_unique_groups, upscaled_unique_groups_indexes = np.unique(upscaled_group_index, return_inverse=True)


total_pixel_colors = np.zeros((*upscaled_unique_groups.shape,3))
np.add.at(
    total_pixel_colors,
    upscaled_unique_groups_indexes,
    rescale(image,(UPSCALE_FACTOR,UPSCALE_FACTOR,1)).reshape(-1,3)
)
total_pixel_counts = np.zeros(upscaled_unique_groups.shape)
np.add.at(
    total_pixel_counts,
    upscaled_unique_groups_indexes,
    1,
)

average_color = total_pixel_colors / total_pixel_counts.reshape(-1,1)


# Map the upscaled pixels to their corresponding Voronoi regions to get colors
#TODO the clip here restricts to square images because it does nto clip the y axis properly
upscaled_pixel_values = average_color[upscaled_unique_groups_indexes].reshape((image.shape[0]*UPSCALE_FACTOR, image.shape[1]*UPSCALE_FACTOR, 3))

# `upscaled_pixel_values` is now the upscaled image with colors sampled from the original image's Voronoi regions

In [ ]:
upscaled_pixel_values.shape

In [ ]:
image.shape

In [ ]:
fig,axs = plt.subplots(1,2)
axs[0].imshow(image)
voronoi_plot_2d(
    Voronoi(voronoi_point_xy[:,::-1]),
    ax=axs[0],
    show_vertices=False,
    show_points=False,
    line_colors='red',
)
axs[1].imshow(upscaled_pixel_values)

for ax in axs:
    #ax.axis("off")
    ax.set_aspect('equal')
    ax.set_xlim([0, image.shape[1]])
    ax.set_ylim([0, image.shape[0]])
axs[1].set_xlim([0, image.shape[1]*UPSCALE_FACTOR])
axs[1].set_ylim([0, image.shape[0]*UPSCALE_FACTOR])
axs[0].invert_yaxis()
axs[1].invert_yaxis()
fig.set_size_inches(30,20)

In [ ]:
#imsave("./data/output/bark_upscaled_2.png", np.round(upscaled_pixel_values*255).astype("u1"))